In [1]:
# !pip install soynlp

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("post.csv",engine = "python",encoding = "utf-8",usecols = ["insta_id","content","label"])
df.head()

,insta_id,content,label
0,hahajuhye,마이삭 조용히지나가길 기원하며 Rain rain go away,0.0
1,hahajuhye,엄마~ 오늘 재하 생일이지? 엄마~ 오늘 재야 생일이디~? 엄마~ 엄ㅁㅏ~ #그래...,0.0
2,hahajuhye,조..좋은언니...2,0.0
3,hahajuhye,좋은언니♡ #흔들리는너의눈동자 #걱정마엄마여기있다,0.0
4,hahajuhye,Look Im serious,0.0


In [4]:
def del_escape(text):
    char_ls = list(text)
    reduced_ls = [char for char in char_ls if "{c!r}".format(c = char).replace("'","") == char]
    return "".join(reduced_ls)

In [5]:
df.dropna(inplace = True)
df["content"] = df["content"].apply(del_escape)
df.dropna(inplace = True)

In [6]:
df["content"].to_csv("post_content.csv",index = False)

In [7]:
from soynlp import DoublespaceLineCorpus

corpus_path = "post_content.csv"
sentences = DoublespaceLineCorpus(corpus_path, iter_sent=True)

In [8]:
from soynlp.word import WordExtractor
from soynlp.noun import LRNounExtractor_v2
noun_extractor = LRNounExtractor_v2()

noun_extractor.train(sentences) # list of str or like
nouns = noun_extractor.extract(min_noun_frequency = 3000)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 6352947 from 2987868 sents. mem=2.521 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=33867543, mem=20.677 Gb
[Noun Extractor] batch prediction was completed for 1064608 words
[Noun Extractor] checked compounds. discovered 1772500 compounds
[Noun Extractor] postprocessing detaching_features : 698 -> 693
[Noun Extractor] postprocessing ignore_features : 693 -> 639
[Noun Extractor] postprocessing ignore_NJ : 639 -> 622
[Noun Extractor] 622 nouns (1772500 compounds) with min frequency=3000
[Noun Extractor] flushing was done. mem=19.649 Gb                    
[Noun Extractor] 25.26 % eojeols are covered


In [9]:
len(nouns)

622

In [10]:
user_seg_df = df.pivot_table(index="insta_id",values=["label"],aggfunc="median")
user_seg_df = user_seg_df.reset_index()
user_seg_df.head()

,insta_id,label
0,0._.do__,1.0
1,0.____.won,1.0
2,000._.bd,0.0
3,00000_min04,0.0
4,00_12.21,0.0


In [11]:
user_text_df = df.pivot_table(index="insta_id",values=["content"],aggfunc="sum")
user_text_df = user_text_df.reset_index()
user_text_df.head()

,insta_id,content
0,0._.do__,약속은 약속이니까✨ ㅋㅋ ㅋ ㅋ 내일도 인증해야지 .. 그래...
1,0.____.won,추억82☆......빨대길이가 왜 다른거야 갑자기 신경쓰여🤨 Photo by @ye...
2,000._.bd,͏s͏i͏m͏p͏l͏e ͏i͏s ͏t͏h͏e ͏b͏e͏s͏t아프로뒷태여신이거 올리래...
3,00000_min04,배구에 다시 반해버렸다Gs 화이팅... 박혜민 선수 화이팅..❤❤Good frien...
4,00_12.21,나 승모근 굉장한 사람이니까 깝치면 다 뒤진다･ﾟ⋆ 🎀 𝒯𝒽𝑒 𝒹𝒶𝓃𝒷𝒾𝓃𝑒 🎀 ...


In [17]:
test_df=pd.read_csv("community.csv")
test_df.head()

,insta_id,content
0,163kg__,"내 주변엔 좋은사람들만 있어서 행복해막내동생이랑🤪내 코 언제 나아,,,초딩아님‼️1..."
1,97s_ba_e,마니 컸다..세탁실 앞에서 자는 그는..하루종일 잔다#사기 #보이스피싱 조심하세요...
2,again._.ty,#다시 #짧은 #글귀 #글 #시 #데일리#사진 #빛바랜 #추억 #칠하기#팔로우좋아요...
3,anyeji620,성남시 청년지원센터해피봄스데이🥳🥳.청년이봄 1주년 축하드립니다><코로나19로 인해 ...
4,clothing.wholesaler,동대문~땡!ㆍ도매언니/밤시장/ 밴드로 초대합니다.아래를 클릭하시면 밴드 가입 가능하...


In [20]:
from sklearn.svm import SVC
pred_ls = list()
for k in range(4):
    dic = {}

    for term in nouns:
        try:
            reduced_term = term.replace("(","").replace(")","")
            dic[reduced_term] = [sum(df.loc[df["label"] == k]["content"].str.contains(reduced_term)),sum(df["label"] == k)-sum(df.loc[df["label"] == k]["content"].str.contains(reduced_term)),
               sum(df.loc[df["label"] != k]["content"].str.contains(reduced_term)),sum(df["label"] != k)-sum(df.loc[df["label"] != k]["content"].str.contains(reduced_term))]
        except:
            pass
    mll_dm = {}
    mll_im = {}
    aic_dm = {}
    aic_im = {}
    relevance = {}
    eps = 1e-12
    for x in dic:
        mll_dm[x] = dic[x][0]*np.log(dic[x][0]+eps)+dic[x][1]*np.log(dic[x][1]+eps)+dic[x][2]*np.log(dic[x][2]+eps)+dic[x][3]*np.log(dic[x][3]+eps)-(len(df))*np.log(len(df))
    for x in dic:
        mll_im[x] = sum(df["label"] == k)*np.log(sum(df["label"] == k))+sum(df["label"] != k)*np.log(sum(df["label"] != k))+(dic[x][0]+dic[x][2])*np.log(dic[x][0]+dic[x][2]+eps)+(dic[x][1]+dic[x][3])*np.log(dic[x][1]+dic[x][3]+eps)-2*(len(df))*np.log(len(df))    
    for x in dic:
        aic_dm[x] = -2*mll_dm[x]+6
    for x in dic:
        aic_im[x] = -2*mll_im[x]+4
    for x in dic:
        if dic[x][0]/(dic[x][0]+dic[x][2]+eps) > dic[x][1]/(dic[x][1]+dic[x][3]+eps):
            relevance[x] = aic_im[x]-aic_dm[x]+2
        else:
            relevance[x] = aic_dm[x]-aic_im[x]-2
    characteristic_term = {}
    i = 0
    for x in relevance:
        if relevance[x] > 0:
            characteristic_term[i] = x
            i += 1
    X = np.array(user_text_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
    y = np.array((user_seg_df["label"] == k).apply(int))
    model = SVC(probability = True)
    model.fit(X,y)
    X_text = np.array(test_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
    pred_ls.append(model.predict_proba(X_text))

In [35]:
predictions = np.array(pred_ls).transpose(1,0,2)[:,:,1].argmax(axis=1)
predictions

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)